In [1]:
from yahooquery import Ticker
import pandas as pd

# Ticker('HRB').income_statement().EBIT/(Ticker('HRB').balance_sheet().TotalAssets-Ticker('HRB').balance_sheet().CurrentLiabilities)

In [29]:
dfResult=pd.DataFrame()

for t in ['VOYA','FNF','FAF','ORI','AFG']:    
    t=t.replace('.','-')
    ticker=Ticker(t)
    balanceSheet=ticker.balance_sheet(frequency="q")
    
    try:
        balanceSheet=balanceSheet[balanceSheet['periodType']=="3M"].sort_values('asOfDate').groupby('symbol').tail(1)
    except: 
        print('balanceSheet error:',balanceSheet)
        continue
        
    incomeStatement=ticker.income_statement(frequency="q")
    try:
        incomeStatement=incomeStatement[incomeStatement['periodType']=="TTM"].sort_values('asOfDate').groupby('symbol').tail(1)
    except:
        print('incomeStatement error:',incomeStatement)
        continue

    try:
        valuationMeasure = pd.DataFrame.from_dict(ticker.summary_detail).loc[['marketCap','dividendYield']].transpose()
        valuationMeasure.index.name = 'symbol'
        valuationMeasure.columns=['MarketCap','dividendYield']
        industry = pd.DataFrame.from_dict(ticker.summary_profile).loc[['industry']].transpose()
        industry.index.name = 'symbol'
        industry.columns=['industry']
    except:
        print('valuationMeasure error:', t)
        continue
    
    mergedBsIs = pd.merge(balanceSheet, incomeStatement, on=['symbol']).fillna(0)
    if mergedBsIs.empty:        
        print('bad join 1:',t)
    elif 'asOfDate_x' in mergedBsIs.columns:
        mergedBsIs.rename(columns={'asOfDate_x':'asOfDate'}, inplace=True)
        
    mergedBsIsSum = pd.merge(mergedBsIs, valuationMeasure, on=['symbol']).fillna(0)
    mergedBsIsSum = pd.merge(mergedBsIsSum, industry, on=['symbol']).fillna(0)
    
    if mergedBsIsSum.empty:
        print('bad join 2:',t)
    
    if 'EBIT' not in mergedBsIsSum.columns:
        print('missing EBIT:', t)
        continue
        
    if 'InvestedCapital' not in mergedBsIsSum.columns:
        print('missing InvestedCapital:', t)
        continue
    
    try:
        cash = mergedBsIsSum.CashAndCashEquivalents
    except:
        print('cash error: ',t)
        continue
    cdl = mergedBsIsSum.CurrentDeferredLiabilities if 'CurrentDeferredLiabilities' in mergedBsIsSum.columns else 0
    ltdclb = mergedBsIsSum.LongTermDebtAndCapitalLeaseObligation if 'LongTermDebtAndCapitalLeaseObligation' in mergedBsIsSum.columns else 0
    ncdl = mergedBsIsSum.NonCurrentDeferredLiabilities if 'NonCurrentDeferredLiabilities' in mergedBsIsSum.columns else 0
    oncl = mergedBsIsSum.OtherNonCurrentLiabilities if 'OtherNonCurrentLiabilities' in mergedBsIsSum.columns else 0
    debt = cdl+ltdclb+ncdl+oncl
    mergedBsIsSum['totalDebt']=debt
    
    try:
        preferredequity = mergedBsIsSum.CapitalStock-mergedBsIsSum.CommonStock
        mergedBsIsSum['preferredequity']=preferredequity
    except:
        print('capital or common stock missing:', t)
        continue
    
    try:
        networth = mergedBsIsSum.InvestedCapital+cash-debt-preferredequity
    except:
        print('InvestedCapital missing:',t)
        continue        
    
    #mergedBsIsSum['roce'] = mergedBsIsSum.EBIT / (mergedBsIsSum.TotalAssets - mergedBsIsSum.CurrentLiabilities)
    mergedBsIsSum['nopat'] = mergedBsIsSum.EBIT*(1-mergedBsIsSum.TaxRateForCalcs)
    mergedBsIsSum['roic'] = mergedBsIsSum.nopat / mergedBsIsSum.InvestedCapital    
    
    dfResult=pd.concat([dfResult,mergedBsIsSum])
dfResult

,asOfDate,periodType_x,currencyCode_x,AccountsReceivable,AdditionalPaidInCapital,CapitalStock,CashAndCashEquivalents,CashCashEquivalentsAndShortTermInvestments,CommonStock,CommonStockEquity,...,SalariesAndWages,LineOfCredit,NonCurrentPensionAndOtherPostretirementBenefitPlans,OtherPayable,OtherReceivables,ReceivablesAdjustmentsAllowances,WriteOff,OtherEquityInterest,OtherEquityAdjustments,UnrealizedGainLoss
symbol,,,,,,,,,,,,,,,,,,,,,
VOYA,2021-09-30,3M,USD,1.358000e+10,1.121500e+10,2000000.0,1.818000e+09,3.743600e+10,2000000.0,8.389000e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FNF,2021-09-30,3M,USD,4.041000e+09,5.790000e+09,0.0,4.454000e+09,3.541600e+10,0.0,9.332000e+09,...,3.459000e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FAF,2021-09-30,3M,USD,4.542270e+08,2.186372e+09,1000.0,2.005106e+09,2.005106e+09,1000.0,5.597135e+09,...,2.257129e+09,0.0,0.0,0.0,6.510480e+08,-13355000.0,-18329000.0,NaN,NaN,NaN
ORI,2021-09-30,3M,USD,6.916100e+09,1.363800e+09,307000000.0,1.334000e+08,1.172180e+10,307000000.0,6.329400e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-87900000.0,NaN,NaN
AFG,2021-09-30,3M,USD,5.015000e+09,1.315000e+09,85000000.0,2.833000e+09,1.326000e+10,85000000.0,5.240000e+09,...,NaN,NaN,NaN,NaN,1.097000e+09,NaN,NaN,0.0,-18000000.0,178000000.0


In [30]:
dfResult[['asOfDate','periodType_y','dividendYield','EBIT','roic','industry']]

,asOfDate,periodType_y,dividendYield,EBIT,roic,industry
symbol,,,,,,
VOYA,2021-09-30,TTM,0.0127,3.175000e+09,0.248262,Financial Conglomerates
FNF,2021-09-30,TTM,0.0350,3.437000e+09,0.219745,Insurance—Specialty
FAF,2021-09-30,TTM,0.0277,1.758784e+09,0.168462,Insurance—Specialty
ORI,2021-09-30,TTM,0.0365,1.836400e+09,0.185455,Insurance—Diversified
AFG,2021-09-30,TTM,0.0170,1.838000e+09,0.204929,Insurance—Property & Casualty


In [26]:
Ticker('VOYA').cash_flow()

,asOfDate,periodType,currencyCode,AmortizationOfSecurities,BeginningCashPosition,CashDividendsPaid,CashFlowFromContinuingFinancingActivities,CashFlowFromContinuingInvestingActivities,CashFlowFromContinuingOperatingActivities,CashFromDiscontinuedFinancingActivities,...,OtherNonCashItems,PreferredStockDividendPaid,PreferredStockIssuance,PurchaseOfBusiness,PurchaseOfInvestment,RepaymentOfDebt,RepurchaseOfCapitalStock,SaleOfBusiness,SaleOfInvestment,StockBasedCompensation
symbol,,,,,,,,,,,,,,,,,,,,,
VOYA,2017-12-31,12M,USD,291000000.0,2.911000e+09,-8000000.0,-7.290000e+08,-1.167000e+09,1.167000e+09,3.840000e+08,...,901000000.0,NaN,NaN,-2.368000e+09,-8.846000e+09,-1.294000e+09,-9.230000e+08,2.283000e+09,8.592000e+09,117000000.0
VOYA,2018-12-31,12M,USD,153000000.0,1.716000e+09,-6000000.0,-5.550000e+08,-3.160000e+08,4.060000e+08,-1.209000e+09,...,-259000000.0,NaN,319000000.0,-1.375000e+09,-9.015000e+09,-1.366000e+09,-1.025000e+09,1.703000e+09,8.335000e+09,96000000.0
VOYA,2019-12-31,12M,USD,95000000.0,1.538000e+09,-72000000.0,-9.340000e+08,-6.290000e+08,1.412000e+09,8.130000e+08,...,578000000.0,-28000000.0,293000000.0,-1.788000e+09,-6.510000e+09,-1.016000e+09,-1.136000e+09,7.910000e+08,6.615000e+09,98000000.0
VOYA,2020-06-30,TTM,USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
VOYA,2020-12-31,12M,USD,247000000.0,1.472000e+09,-112000000.0,1.179000e+09,-1.957000e+09,1.617000e+09,5.230000e+08,...,706000000.0,-36000000.0,0.0,-1.453000e+09,-6.911000e+09,-9.910000e+08,-5.160000e+08,7.460000e+08,5.614000e+09,88000000.0
VOYA,2021-09-30,TTM,USD,NaN,1.549000e+09,-113000000.0,2.510000e+08,-9.390000e+08,7.570000e+08,1.270000e+08,...,NaN,-36000000.0,NaN,-2.123000e+09,-7.577000e+09,-1.088000e+09,-9.530000e+08,1.914000e+09,6.271000e+09,NaN
